# YOLOv3 using MMdetection on Stanford Cars Dataset

In [1]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [2]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 704.4MB 27kB/s 
     |████████████████████████████████| 6.7MB 19.0MB/s 
ERROR: torchtext 0.10.0 has requirement torch==1.9.0, but you'll have torch 1.5.1+cu101 which is incompatible.
  Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successfully uninstalled torchvision-0.10.0+cu102


In [4]:
#***************************************************
#  Hangs for 10 min while building wheel but it's ok
#***************************************************
# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full

# Install mmdetection
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install -e .

# install Pillow 7.0.0 back in order to avoid bug in colab
!pip install Pillow==7.0.0


     |████████████████████████████████| 317kB 24.1MB/s 
     |████████████████████████████████| 194kB 54.1MB/s 
  Created wheel for mmcv-full: filename=mmcv_full-1.3.9-cp37-cp37m-linux_x86_64.whl size=25575930 sha256=f2396956276cf703ee978894e92bf3beb42a84ee0f59579e659da47b9d979ae7
  Stored in directory: /root/.cache/pip/wheels/e0/2b/d5/410850bd0f29e241507daf354083142a0c25d66667f22e707a
Successfully built mmcv-full
Cloning into 'mmdetection'...
remote: Enumerating objects: 19286, done.
remote: Counting objects: 100% (224/224), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 19286 (delta 95), reused 190 (delta 84), pack-reused 19062
Receiving objects: 100% (19286/19286), 23.57 MiB | 33.02 MiB/s, done.
Resolving deltas: 100% (13496/13496), done.
/content/mmdetection
Obtaining file:///content/mmdetection
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=832d8df2864506523c07e1b01aac4ad11bed703a27db7f59f67daaa4dc20248

In [1]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.5.1+cu101 True
2.14.0
11.0
GCC 7.5


In [2]:
#***************************************
# Dataset with all images 000001 - 016185.jpg
#***************************************
!mkdir data
!wget -c http://ai.stanford.edu/~jkrause/car196/car_ims.tgz \
      -O cars_all.tgz

--2021-07-12 15:43:38--  http://ai.stanford.edu/~jkrause/car196/car_ims.tgz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1956628579 (1.8G) [application/x-gzip]
Saving to: ‘cars_all.tgz’

cars_all.tgz        100%[===================>]   1.82G  18.8MB/s    in 1m 42s  

2021-07-12 15:45:20 (18.2 MB/s) - ‘cars_all.tgz’ saved [1956628579/1956628579]



In [3]:
#***************************************
# Extracting all images to the data folder (from previous step)
#***************************************
!tar -xvf  '/content/cars_all.tgz' -C '/content/data'

Streaming output truncated to the last 5000 lines.
car_ims/006903.jpg
car_ims/006571.jpg
car_ims/011511.jpg
car_ims/011963.jpg
car_ims/006048.jpg
car_ims/003716.jpg
car_ims/010351.jpg
car_ims/010884.jpg
car_ims/002156.jpg
car_ims/007608.jpg
car_ims/013244.jpg
car_ims/013991.jpg
car_ims/001043.jpg
car_ims/012876.jpg
car_ims/012404.jpg
car_ims/000603.jpg
car_ims/010730.jpg
car_ims/002290.jpg
car_ims/010097.jpg
car_ims/007269.jpg
car_ims/002945.jpg
car_ims/002537.jpg
car_ims/011170.jpg
car_ims/003377.jpg
car_ims/006429.jpg
car_ims/014189.jpg
car_ims/012065.jpg
car_ims/000262.jpg
car_ims/009929.jpg
car_ims/013625.jpg
car_ims/001385.jpg
car_ims/013182.jpg
car_ims/001422.jpg
car_ims/001850.jpg
car_ims/004878.jpg
car_ims/001354.jpg
car_ims/001881.jpg
car_ims/013153.jpg
car_ims/000514.jpg
car_ims/000966.jpg
car_ims/012713.jpg
car_ims/003873.jpg
car_ims/003401.jpg
car_ims/011606.jpg
car_ims/014158.jpg
car_ims/002241.jpg
car_ims/002994.jpg
car_ims/015718.jpg
car_ims/010046.jpg
car_ims/000175.jpg

In [5]:
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset

In [6]:
import itertools
import logging
import os.path as osp
import tempfile
import warnings
from collections import OrderedDict

import numpy as np
from mmcv.utils import print_log
from terminaltables import AsciiTable

from mmdet.core import eval_recalls
from mmdet.datasets.api_wrappers import COCO, COCOeval

In [44]:
#***************************************
# Registering a custom "CarDataset" as customdataset like COCO.py | Classes need to be changed if required.
#***************************************

@DATASETS.register_module()
class CarDataset(CustomDataset):

  CLASSES = ("AM_General_Hummer_SUV_2000","Acura_Integra_Type_R_2001","Acura_RL_Sedan_2012","Acura_TL_Sedan_2012",
             "Acura_TL_Type-S_2008","Acura_TSX_Sedan_2012","Acura_ZDX_Hatchback_2012","Aston_Martin_V8_Vantage_Convertible_2012",
             "Aston_Martin_V8_Vantage_Coupe_2012","Aston_Martin_Virage_Convertible_2012","Aston_Martin_Virage_Coupe_2012","Audi_100_Sedan_1994",
             "Audi_100_Wagon_1994","Audi_A5_Coupe_2012","Audi_R8_Coupe_2012","Audi_RS_4_Convertible_2008","Audi_S4_Sedan_2007","Audi_S4_Sedan_2012",
             "Audi_S5_Convertible_2012","Audi_S5_Coupe_2012","Audi_S6_Sedan_2011","Audi_TTS_Coupe_2012","Audi_TT_Hatchback_2011","Audi_TT_RS_Coupe_2012",
             "Audi_V8_Sedan_1994","BMW_1_Series_Convertible_2012","BMW_1_Series_Coupe_2012","BMW_3_Series_Sedan_2012","BMW_3_Series_Wagon_2012",
             "BMW_6_Series_Convertible_2007","BMW_ActiveHybrid_5_Sedan_2012","BMW_M3_Coupe_2012","BMW_M5_Sedan_2010","BMW_M6_Convertible_2010",
             "BMW_X3_SUV_2012","BMW_X5_SUV_2007","BMW_X6_SUV_2012","BMW_Z4_Convertible_2012","Bentley_Arnage_Sedan_2009","Bentley_Continental_Flying_Spur_Sedan_2007",
             "Bentley_Continental_GT_Coupe_2007","Bentley_Continental_GT_Coupe_2012","Bentley_Continental_Supersports_Conv._Convertible_2012",
             "Bentley_Mulsanne_Sedan_2011","Bugatti_Veyron_16.4_Convertible_2009","Bugatti_Veyron_16.4_Coupe_2009","Buick_Enclave_SUV_2012",
             "Buick_Rainier_SUV_2007","Buick_Regal_GS_2012","Buick_Verano_Sedan_2012","Cadillac_CTS-V_Sedan_2012","Cadillac_Escalade_EXT_Crew_Cab_2007",
             "Cadillac_SRX_SUV_2012","Chevrolet_Avalanche_Crew_Cab_2012","Chevrolet_Camaro_Convertible_2012","Chevrolet_Cobalt_SS_2010",
             "Chevrolet_Corvette_Convertible_2012","Chevrolet_Corvette_Ron_Fellows_Edition_Z06_2007","Chevrolet_Corvette_ZR1_2012",
             "Chevrolet_Express_Cargo_Van_2007","Chevrolet_Express_Van_2007","Chevrolet_HHR_SS_2010","Chevrolet_Impala_Sedan_2007",
             "Chevrolet_Malibu_Hybrid_Sedan_2010","Chevrolet_Malibu_Sedan_2007","Chevrolet_Monte_Carlo_Coupe_2007","Chevrolet_Silverado_1500_Classic_Extended_Cab_2007",
             "Chevrolet_Silverado_1500_Extended_Cab_2012","Chevrolet_Silverado_1500_Hybrid_Crew_Cab_2012","Chevrolet_Silverado_1500_Regular_Cab_2012",
             "Chevrolet_Silverado_2500HD_Regular_Cab_2012","Chevrolet_Sonic_Sedan_2012","Chevrolet_Tahoe_Hybrid_SUV_2012","Chevrolet_TrailBlazer_SS_2009",
             "Chevrolet_Traverse_SUV_2012","Chrysler_300_SRT-8_2010","Chrysler_Aspen_SUV_2009","Chrysler_Crossfire_Convertible_2008",
             "Chrysler_PT_Cruiser_Convertible_2008","Chrysler_Sebring_Convertible_2010","Chrysler_Town_and_Country_Minivan_2012","Daewoo_Nubira_Wagon_2002",
             "Dodge_Caliber_Wagon_2007","Dodge_Caliber_Wagon_2012","Dodge_Caravan_Minivan_1997","Dodge_Challenger_SRT8_2011","Dodge_Charger_SRT-8_2009",
             "Dodge_Charger_Sedan_2012","Dodge_Dakota_Club_Cab_2007","Dodge_Dakota_Crew_Cab_2010","Dodge_Durango_SUV_2007","Dodge_Durango_SUV_2012",
             "Dodge_Journey_SUV_2012","Dodge_Magnum_Wagon_2008","Dodge_Ram_Pickup_3500_Crew_Cab_2010","Dodge_Ram_Pickup_3500_Quad_Cab_2009",
             "Dodge_Sprinter_Cargo_Van_2009","Eagle_Talon_Hatchback_1998","FIAT_500_Abarth_2012","FIAT_500_Convertible_2012","Ferrari_458_Italia_Convertible_2012",
             "Ferrari_458_Italia_Coupe_2012","Ferrari_California_Convertible_2012","Ferrari_FF_Coupe_2012","Fisker_Karma_Sedan_2012","Ford_E-Series_Wagon_Van_2012",
             "Ford_Edge_SUV_2012","Ford_Expedition_EL_SUV_2009","Ford_F-150_Regular_Cab_2007","Ford_F-150_Regular_Cab_2012","Ford_F-450_Super_Duty_Crew_Cab_2012",
             "Ford_Fiesta_Sedan_2012","Ford_Focus_Sedan_2007","Ford_Freestar_Minivan_2007","Ford_GT_Coupe_2006","Ford_Mustang_Convertible_2007",
             "Ford_Ranger_SuperCab_2011","GMC_Acadia_SUV_2012","GMC_Canyon_Extended_Cab_2012","GMC_Savana_Van_2012","GMC_Terrain_SUV_2012",
             "GMC_Yukon_Hybrid_SUV_2012","Geo_Metro_Convertible_1993","HUMMER_H2_SUT_Crew_Cab_2009","HUMMER_H3T_Crew_Cab_2010","Honda_Accord_Coupe_2012",
             "Honda_Accord_Sedan_2012","Honda_Odyssey_Minivan_2007","Honda_Odyssey_Minivan_2012","Hyundai_Accent_Sedan_2012","Hyundai_Azera_Sedan_2012",
             "Hyundai_Elantra_Sedan_2007","Hyundai_Elantra_Touring_Hatchback_2012","Hyundai_Genesis_Sedan_2012","Hyundai_Santa_Fe_SUV_2012",
             "Hyundai_Sonata_Hybrid_Sedan_2012","Hyundai_Sonata_Sedan_2012","Hyundai_Tucson_SUV_2012","Hyundai_Veloster_Hatchback_2012","Hyundai_Veracruz_SUV_2012",
             "Infiniti_G_Coupe_IPL_2012","Infiniti_QX56_SUV_2011","Isuzu_Ascender_SUV_2008","Jaguar_XK_XKR_2012","Jeep_Compass_SUV_2012",
             "Jeep_Grand_Cherokee_SUV_2012","Jeep_Liberty_SUV_2012","Jeep_Patriot_SUV_2012","Jeep_Wrangler_SUV_2012","Lamborghini_Aventador_Coupe_2012",
             "Lamborghini_Diablo_Coupe_2001","Lamborghini_Gallardo_LP_570-4_Superleggera_2012","Lamborghini_Reventon_Coupe_2008","Land_Rover_LR2_SUV_2012",
             "Land_Rover_Range_Rover_SUV_2012","Lincoln_Town_Car_Sedan_2011","MINI_Cooper_Roadster_Convertible_2012","Maybach_Landaulet_Convertible_2012",
             "Mazda_Tribute_SUV_2011","McLaren_MP4-12C_Coupe_2012","Mercedes-Benz_300-Class_Convertible_1993","Mercedes-Benz_C-Class_Sedan_2012",
             "Mercedes-Benz_E-Class_Sedan_2012","Mercedes-Benz_S-Class_Sedan_2012","Mercedes-Benz_SL-Class_Coupe_2009","Mercedes-Benz_Sprinter_Van_2012",
             "Mitsubishi_Lancer_Sedan_2012","Nissan_240SX_Coupe_1998","Nissan_Juke_Hatchback_2012","Nissan_Leaf_Hatchback_2012","Nissan_NV_Passenger_Van_2012",
             "Plymouth_Neon_Coupe_1999","Porsche_Panamera_Sedan_2012","Ram_C/V_Cargo_Van_Minivan_2012","Rolls-Royce_Ghost_Sedan_2012",
             "Rolls-Royce_Phantom_Drophead_Coupe_Convertible_2012","Rolls-Royce_Phantom_Sedan_2012","Scion_xD_Hatchback_2012","Spyker_C8_Convertible_2009",
             "Spyker_C8_Coupe_2009","Suzuki_Aerio_Sedan_2007","Suzuki_Kizashi_Sedan_2012","Suzuki_SX4_Hatchback_2012","Suzuki_SX4_Sedan_2012",
             "Tesla_Model_S_Sedan_2012","Toyota_4Runner_SUV_2012","Toyota_Camry_Sedan_2012","Toyota_Corolla_Sedan_2012","Toyota_Sequoia_SUV_2012",
             "Volkswagen_Beetle_Hatchback_2012","Volkswagen_Golf_Hatchback_1991","Volkswagen_Golf_Hatchback_2012","Volvo_240_Sedan_1993",
             "Volvo_C30_Hatchback_2012","Volvo_XC90_SUV_2007","smart_fortwo_Convertible_2012" )
  
  def load_annotations(self, ann_file):
        """Load annotation from COCO style annotation file.
        Args:
            ann_file (str): Path of annotation file.
        Returns:
            list[dict]: Annotation info from COCO api.
        """

        self.coco = COCO(ann_file)
        # The order of returned `cat_ids` will not
        # change with the order of the CLASSES
        self.cat_ids = self.coco.get_cat_ids(cat_names=self.CLASSES)

        self.cat2label = {cat_id: i for i, cat_id in enumerate(self.cat_ids)}
        self.img_ids = self.coco.get_img_ids()
        data_infos = []
        total_ann_ids = []
        for i in self.img_ids:
            info = self.coco.load_imgs([i])[0]
            info['filename'] = info['file_name']
            data_infos.append(info)
            ann_ids = self.coco.get_ann_ids(img_ids=[i])
            total_ann_ids.extend(ann_ids)
        assert len(set(total_ann_ids)) == len(
            total_ann_ids), f"Annotation ids in '{ann_file}' are not unique!"
        return data_infos
    
  def get_ann_info(self, idx):
        """Get COCO annotation by index.
        Args:
            idx (int): Index of data.
        Returns:
            dict: Annotation info of specified index.
        """

        img_id = self.data_infos[idx]['id']
        ann_ids = self.coco.get_ann_ids(img_ids=[img_id])
        ann_info = self.coco.load_anns(ann_ids)
        return self._parse_ann_info(self.data_infos[idx], ann_info)

  def get_cat_ids(self, idx):
        """Get COCO category ids by index.
        Args:
            idx (int): Index of data.
        Returns:
            list[int]: All categories in the image of specified index.
        """

        img_id = self.data_infos[idx]['id']
        ann_ids = self.coco.get_ann_ids(img_ids=[img_id])
        ann_info = self.coco.load_anns(ann_ids)
        return [ann['category_id'] for ann in ann_info]

  def _filter_imgs(self, min_size=32):
        """Filter images too small or without ground truths."""
        valid_inds = []
        # obtain images that contain annotation
        ids_with_ann = set(_['image_id'] for _ in self.coco.anns.values())
        # obtain images that contain annotations of the required categories
        ids_in_cat = set()
        for i, class_id in enumerate(self.cat_ids):
            ids_in_cat |= set(self.coco.cat_img_map[class_id])
        # merge the image id sets of the two conditions and use the merged set
        # to filter out images if self.filter_empty_gt=True
        ids_in_cat &= ids_with_ann

        valid_img_ids = []
        for i, img_info in enumerate(self.data_infos):
            img_id = self.img_ids[i]
            if self.filter_empty_gt and img_id not in ids_in_cat:
                continue
            if min(img_info['width'], img_info['height']) >= min_size:
                valid_inds.append(i)
                valid_img_ids.append(img_id)
        self.img_ids = valid_img_ids
        return valid_inds

  def _parse_ann_info(self, img_info, ann_info):
        """Parse bbox and mask annotation.
        Args:
            ann_info (list[dict]): Annotation info of an image.
            with_mask (bool): Whether to parse mask annotations.
        Returns:
            dict: A dict containing the following keys: bboxes, bboxes_ignore,\
                labels, masks, seg_map. "masks" are raw annotations and not \
                decoded into binary masks.
        """
        gt_bboxes = []
        gt_labels = []
        gt_bboxes_ignore = []
        gt_masks_ann = []
        for i, ann in enumerate(ann_info):
            if ann.get('ignore', False):
                continue
            x1, y1, w, h = ann['bbox']
            inter_w = max(0, min(x1 + w, img_info['width']) - max(x1, 0))
            inter_h = max(0, min(y1 + h, img_info['height']) - max(y1, 0))
            if inter_w * inter_h == 0:
                continue
            if ann['area'] <= 0 or w < 1 or h < 1:
                continue
            if ann['category_id'] not in self.cat_ids:
                continue
            bbox = [x1, y1, x1 + w, y1 + h]
            if ann.get('iscrowd', False):
                gt_bboxes_ignore.append(bbox)
            else:
                gt_bboxes.append(bbox)
                gt_labels.append(self.cat2label[ann['category_id']])
                gt_masks_ann.append(ann.get('segmentation', None))

        if gt_bboxes:
            gt_bboxes = np.array(gt_bboxes, dtype=np.float32)
            gt_labels = np.array(gt_labels, dtype=np.int64)
        else:
            gt_bboxes = np.zeros((0, 4), dtype=np.float32)
            gt_labels = np.array([], dtype=np.int64)

        if gt_bboxes_ignore:
            gt_bboxes_ignore = np.array(gt_bboxes_ignore, dtype=np.float32)
        else:
            gt_bboxes_ignore = np.zeros((0, 4), dtype=np.float32)

        seg_map = img_info['filename'].replace('jpg', 'png')

        ann = dict(
            bboxes=gt_bboxes,
            labels=gt_labels,
            bboxes_ignore=gt_bboxes_ignore,
            masks=gt_masks_ann,
            seg_map=seg_map)

        return ann

  def xyxy2xywh(self, bbox):
        """Convert ``xyxy`` style bounding boxes to ``xywh`` style for COCO
        evaluation.
        Args:
            bbox (numpy.ndarray): The bounding boxes, shape (4, ), in
                ``xyxy`` order.
        Returns:
            list[float]: The converted bounding boxes, in ``xywh`` order.
        """

        _bbox = bbox.tolist()
        return [
            _bbox[0],
            _bbox[1],
            _bbox[2] - _bbox[0],
            _bbox[3] - _bbox[1],
        ]

  def _proposal2json(self, results):
        """Convert proposal results to COCO json style."""
        json_results = []
        for idx in range(len(self)):
            img_id = self.img_ids[idx]
            bboxes = results[idx]
            for i in range(bboxes.shape[0]):
                data = dict()
                data['image_id'] = img_id
                data['bbox'] = self.xyxy2xywh(bboxes[i])
                data['score'] = float(bboxes[i][4])
                data['category_id'] = 1
                json_results.append(data)
        return json_results

  def _det2json(self, results):
        """Convert detection results to COCO json style."""
        json_results = []
        for idx in range(len(self)):
            img_id = self.img_ids[idx]
            result = results[idx]
            for label in range(len(result)):
                bboxes = result[label]
                for i in range(bboxes.shape[0]):
                    data = dict()
                    data['image_id'] = img_id
                    data['bbox'] = self.xyxy2xywh(bboxes[i])
                    data['score'] = float(bboxes[i][4])
                    data['category_id'] = self.cat_ids[label]
                    json_results.append(data)
        return json_results

  def _segm2json(self, results):
        """Convert instance segmentation results to COCO json style."""
        bbox_json_results = []
        segm_json_results = []
        for idx in range(len(self)):
            img_id = self.img_ids[idx]
            det, seg = results[idx]
            for label in range(len(det)):
                # bbox results
                bboxes = det[label]
                for i in range(bboxes.shape[0]):
                    data = dict()
                    data['image_id'] = img_id
                    data['bbox'] = self.xyxy2xywh(bboxes[i])
                    data['score'] = float(bboxes[i][4])
                    data['category_id'] = self.cat_ids[label]
                    bbox_json_results.append(data)

                # segm results
                # some detectors use different scores for bbox and mask
                if isinstance(seg, tuple):
                    segms = seg[0][label]
                    mask_score = seg[1][label]
                else:
                    segms = seg[label]
                    mask_score = [bbox[4] for bbox in bboxes]
                for i in range(bboxes.shape[0]):
                    data = dict()
                    data['image_id'] = img_id
                    data['bbox'] = self.xyxy2xywh(bboxes[i])
                    data['score'] = float(mask_score[i])
                    data['category_id'] = self.cat_ids[label]
                    if isinstance(segms[i]['counts'], bytes):
                        segms[i]['counts'] = segms[i]['counts'].decode()
                    data['segmentation'] = segms[i]
                    segm_json_results.append(data)
        return bbox_json_results, segm_json_results

  def results2json(self, results, outfile_prefix):
        """Dump the detection results to a COCO style json file.
        There are 3 types of results: proposals, bbox predictions, mask
        predictions, and they have different data types. This method will
        automatically recognize the type, and dump them to json files.
        Args:
            results (list[list | tuple | ndarray]): Testing results of the
                dataset.
            outfile_prefix (str): The filename prefix of the json files. If the
                prefix is "somepath/xxx", the json files will be named
                "somepath/xxx.bbox.json", "somepath/xxx.segm.json",
                "somepath/xxx.proposal.json".
        Returns:
            dict[str: str]: Possible keys are "bbox", "segm", "proposal", and \
                values are corresponding filenames.
        """
        result_files = dict()
        if isinstance(results[0], list):
            json_results = self._det2json(results)
            result_files['bbox'] = f'{outfile_prefix}.bbox.json'
            result_files['proposal'] = f'{outfile_prefix}.bbox.json'
            mmcv.dump(json_results, result_files['bbox'])
        elif isinstance(results[0], tuple):
            json_results = self._segm2json(results)
            result_files['bbox'] = f'{outfile_prefix}.bbox.json'
            result_files['proposal'] = f'{outfile_prefix}.bbox.json'
            result_files['segm'] = f'{outfile_prefix}.segm.json'
            mmcv.dump(json_results[0], result_files['bbox'])
            mmcv.dump(json_results[1], result_files['segm'])
        elif isinstance(results[0], np.ndarray):
            json_results = self._proposal2json(results)
            result_files['proposal'] = f'{outfile_prefix}.proposal.json'
            mmcv.dump(json_results, result_files['proposal'])
        else:
            raise TypeError('invalid type of results')
        return result_files

  def fast_eval_recall(self, results, proposal_nums, iou_thrs, logger=None):
        gt_bboxes = []
        for i in range(len(self.img_ids)):
            ann_ids = self.coco.get_ann_ids(img_ids=self.img_ids[i])
            ann_info = self.coco.load_anns(ann_ids)
            if len(ann_info) == 0:
                gt_bboxes.append(np.zeros((0, 4)))
                continue
            bboxes = []
            for ann in ann_info:
                if ann.get('ignore', False) or ann['iscrowd']:
                    continue
                x1, y1, w, h = ann['bbox']
                bboxes.append([x1, y1, x1 + w, y1 + h])
            bboxes = np.array(bboxes, dtype=np.float32)
            if bboxes.shape[0] == 0:
                bboxes = np.zeros((0, 4))
            gt_bboxes.append(bboxes)

        recalls = eval_recalls(
            gt_bboxes, results, proposal_nums, iou_thrs, logger=logger)
        ar = recalls.mean(axis=1)
        return ar

  def format_results(self, results, jsonfile_prefix=None, **kwargs):
        """Format the results to json (standard format for COCO evaluation).
        Args:
            results (list[tuple | numpy.ndarray]): Testing results of the
                dataset.
            jsonfile_prefix (str | None): The prefix of json files. It includes
                the file path and the prefix of filename, e.g., "a/b/prefix".
                If not specified, a temp file will be created. Default: None.
        Returns:
            tuple: (result_files, tmp_dir), result_files is a dict containing \
                the json filepaths, tmp_dir is the temporal directory created \
                for saving json files when jsonfile_prefix is not specified.
        """
        assert isinstance(results, list), 'results must be a list'
        assert len(results) == len(self), (
            'The length of results is not equal to the dataset len: {} != {}'.
            format(len(results), len(self)))

        if jsonfile_prefix is None:
            tmp_dir = tempfile.TemporaryDirectory()
            jsonfile_prefix = osp.join(tmp_dir.name, 'results')
        else:
            tmp_dir = None
        result_files = self.results2json(results, jsonfile_prefix)
        return result_files, tmp_dir

  def evaluate(self,
                 results,
                 metric='bbox',
                 logger=None,
                 jsonfile_prefix=None,
                 classwise=False,
                 proposal_nums=(100, 300, 1000),
                 iou_thrs=None,
                 metric_items=None):
        """Evaluation in COCO protocol.
        Args:
            results (list[list | tuple]): Testing results of the dataset.
            metric (str | list[str]): Metrics to be evaluated. Options are
                'bbox', 'segm', 'proposal', 'proposal_fast'.
            logger (logging.Logger | str | None): Logger used for printing
                related information during evaluation. Default: None.
            jsonfile_prefix (str | None): The prefix of json files. It includes
                the file path and the prefix of filename, e.g., "a/b/prefix".
                If not specified, a temp file will be created. Default: None.
            classwise (bool): Whether to evaluating the AP for each class.
            proposal_nums (Sequence[int]): Proposal number used for evaluating
                recalls, such as recall@100, recall@1000.
                Default: (100, 300, 1000).
            iou_thrs (Sequence[float], optional): IoU threshold used for
                evaluating recalls/mAPs. If set to a list, the average of all
                IoUs will also be computed. If not specified, [0.50, 0.55,
                0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95] will be used.
                Default: None.
            metric_items (list[str] | str, optional): Metric items that will
                be returned. If not specified, ``['AR@100', 'AR@300',
                'AR@1000', 'AR_s@1000', 'AR_m@1000', 'AR_l@1000' ]`` will be
                used when ``metric=='proposal'``, ``['mAP', 'mAP_50', 'mAP_75',
                'mAP_s', 'mAP_m', 'mAP_l']`` will be used when
                ``metric=='bbox' or metric=='segm'``.
        Returns:
            dict[str, float]: COCO style evaluation metric.
        """

        metrics = metric if isinstance(metric, list) else [metric]
        allowed_metrics = ['bbox', 'segm', 'proposal', 'proposal_fast']
        for metric in metrics:
            if metric not in allowed_metrics:
                raise KeyError(f'metric {metric} is not supported')
        if iou_thrs is None:
            iou_thrs = np.linspace(
                .5, 0.95, int(np.round((0.95 - .5) / .05)) + 1, endpoint=True)
        if metric_items is not None:
            if not isinstance(metric_items, list):
                metric_items = [metric_items]

        result_files, tmp_dir = self.format_results(results, jsonfile_prefix)

        eval_results = OrderedDict()
        cocoGt = self.coco
        for metric in metrics:
            msg = f'Evaluating {metric}...'
            if logger is None:
                msg = '\n' + msg
            print_log(msg, logger=logger)

            if metric == 'proposal_fast':
                ar = self.fast_eval_recall(
                    results, proposal_nums, iou_thrs, logger='silent')
                log_msg = []
                for i, num in enumerate(proposal_nums):
                    eval_results[f'AR@{num}'] = ar[i]
                    log_msg.append(f'\nAR@{num}\t{ar[i]:.4f}')
                log_msg = ''.join(log_msg)
                print_log(log_msg, logger=logger)
                continue

            iou_type = 'bbox' if metric == 'proposal' else metric
            if metric not in result_files:
                raise KeyError(f'{metric} is not in results')
            try:
                predictions = mmcv.load(result_files[metric])
                if iou_type == 'segm':
                    # Refer to https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocotools/coco.py#L331  # noqa
                    # When evaluating mask AP, if the results contain bbox,
                    # cocoapi will use the box area instead of the mask area
                    # for calculating the instance area. Though the overall AP
                    # is not affected, this leads to different
                    # small/medium/large mask AP results.
                    for x in predictions:
                        x.pop('bbox')
                    warnings.simplefilter('once')
                    warnings.warn(
                        'The key "bbox" is deleted for more accurate mask AP '
                        'of small/medium/large instances since v2.12.0. This '
                        'does not change the overall mAP calculation.',
                        UserWarning)
                cocoDt = cocoGt.loadRes(predictions)
            except IndexError:
                print_log(
                    'The testing results of the whole dataset is empty.',
                    logger=logger,
                    level=logging.ERROR)
                break

            cocoEval = COCOeval(cocoGt, cocoDt, iou_type)
            cocoEval.params.catIds = self.cat_ids
            cocoEval.params.imgIds = self.img_ids
            cocoEval.params.maxDets = list(proposal_nums)
            cocoEval.params.iouThrs = iou_thrs
            # mapping of cocoEval.stats
            coco_metric_names = {
                'mAP': 0,
                'mAP_50': 1,
                'mAP_75': 2,
                'mAP_s': 3,
                'mAP_m': 4,
                'mAP_l': 5,
                'AR@100': 6,
                'AR@300': 7,
                'AR@1000': 8,
                'AR_s@1000': 9,
                'AR_m@1000': 10,
                'AR_l@1000': 11
            }
            if metric_items is not None:
                for metric_item in metric_items:
                    if metric_item not in coco_metric_names:
                        raise KeyError(
                            f'metric item {metric_item} is not supported')

            if metric == 'proposal':
                cocoEval.params.useCats = 0
                cocoEval.evaluate()
                cocoEval.accumulate()
                cocoEval.summarize()
                if metric_items is None:
                    metric_items = [
                        'AR@100', 'AR@300', 'AR@1000', 'AR_s@1000',
                        'AR_m@1000', 'AR_l@1000'
                    ]

                for item in metric_items:
                    val = float(
                        f'{cocoEval.stats[coco_metric_names[item]]:.3f}')
                    eval_results[item] = val
            else:
                cocoEval.evaluate()
                cocoEval.accumulate()
                cocoEval.summarize()
                if classwise:  # Compute per-category AP
                    # Compute per-category AP
                    # from https://github.com/facebookresearch/detectron2/
                    precisions = cocoEval.eval['precision']
                    # precision: (iou, recall, cls, area range, max dets)
                    assert len(self.cat_ids) == precisions.shape[2]

                    results_per_category = []
                    for idx, catId in enumerate(self.cat_ids):
                        # area range index 0: all area ranges
                        # max dets index -1: typically 100 per image
                        nm = self.coco.loadCats(catId)[0]
                        precision = precisions[:, :, idx, 0, -1]
                        precision = precision[precision > -1]
                        if precision.size:
                            ap = np.mean(precision)
                        else:
                            ap = float('nan')
                        results_per_category.append(
                            (f'{nm["name"]}', f'{float(ap):0.3f}'))

                    num_columns = min(6, len(results_per_category) * 2)
                    results_flatten = list(
                        itertools.chain(*results_per_category))
                    headers = ['category', 'AP'] * (num_columns // 2)
                    results_2d = itertools.zip_longest(*[
                        results_flatten[i::num_columns]
                        for i in range(num_columns)
                    ])
                    table_data = [headers]
                    table_data += [result for result in results_2d]
                    table = AsciiTable(table_data)
                    print_log('\n' + table.table, logger=logger)

                if metric_items is None:
                    metric_items = [
                        'mAP', 'mAP_50', 'mAP_75', 'mAP_s', 'mAP_m', 'mAP_l'
                    ]

                for metric_item in metric_items:
                    key = f'{metric}_{metric_item}'
                    val = float(
                        f'{cocoEval.stats[coco_metric_names[metric_item]]:.3f}'
                    )
                    eval_results[key] = val
                ap = cocoEval.stats[:6]
                eval_results[f'{metric}_mAP_copypaste'] = (
                    f'{ap[0]:.3f} {ap[1]:.3f} {ap[2]:.3f} {ap[3]:.3f} '
                    f'{ap[4]:.3f} {ap[5]:.3f}')
        if tmp_dir is not None:
            tmp_dir.cleanup()
        return eval_results     

  

KeyError: ignored

In [45]:
#***************************************
# Downloading model checkpoints to local folder called 'checkpoints'
#***************************************

!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmdetection/v2.0/yolo/yolov3_d53_mstrain-608_273e_coco/yolov3_d53_mstrain-608_273e_coco_20210518_115020-a2c3acb8.pth \
      -O checkpoints/yolov3_d53_mstrain-608_273e_coco_20210518_115020-a2c3acb8.pth

mkdir: cannot create directory ‘checkpoints’: File exists
--2021-07-12 16:28:19--  https://download.openmmlab.com/mmdetection/v2.0/yolo/yolov3_d53_mstrain-608_273e_coco/yolov3_d53_mstrain-608_273e_coco_20210518_115020-a2c3acb8.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.252.96.35
Connecting to download.openmmlab.com (download.openmmlab.com)|47.252.96.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248093138 (237M) [application/octet-stream]
Saving to: ‘checkpoints/yolov3_d53_mstrain-608_273e_coco_20210518_115020-a2c3acb8.pth’

checkpoints/yolov3_ 100%[===================>] 236.60M  9.14MB/s    in 27s     

2021-07-12 16:28:48 (8.92 MB/s) - ‘checkpoints/yolov3_d53_mstrain-608_273e_coco_20210518_115020-a2c3acb8.pth’ saved [248093138/248093138]



In [64]:
#***************************************
# Import config for the required model from model zoo
#***************************************
import mmcv
from mmcv import Config
cfg = Config.fromfile('./mmdetection/configs/yolo/yolov3_d53_mstrain-608_273e_coco.py')

In [67]:
#***************************************
# All configs according to corresponding model zoo config file. Errors here : refer config file
#***************************************

from mmdet.apis import set_random_seed

# Modify dataset type and path
cfg.dataset_type = 'CarDataset'
cfg.data_root = '/content/data/'

cfg.data.test.type = 'CarDataset'
cfg.data.test.data_root = '/content/data/'
cfg.data.test.ann_file = '/content/data/annotations/instances_train.json'
cfg.data.test.img_prefix = 'car_ims'

cfg.data.train.type = 'CarDataset'
cfg.data.train.data_root = '/content/data/'
cfg.data.train.ann_file = '/content/data/annotations/instances_test.json'
cfg.data.train.img_prefix = 'car_ims'

cfg.data.val.type = 'CarDataset'
cfg.data.val.data_root = '/content/data/'
cfg.data.val.ann_file = '/content/data/annotations/instances_val.json'
cfg.data.val.img_prefix = 'car_ims'

# modify num classes of the model in box head
#cfg.model.roi_head.bbox_head.num_classes = 196
cfg.model.bbox_head.num_classes = 196
# We can still use the pre-trained Mask RCNN model though we do not need to
# use the mask branch
cfg.load_from = 'checkpoints/yolov3_d53_mstrain-608_273e_coco_20210518_115020-a2c3acb8.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './frcnn_train'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.02 / 8
#cfg.lr_config.warmup = None
cfg.lr_config.policy = "step"
cfg.lr_config.warmup = "linear"
cfg.log_config.interval = 10

# Change the evaluation metric since we use customized dataset.
#cfg.evaluation.metric = 'mAP'
cfg.evaluation.metric = 'bbox'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 12
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 12

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)


# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

Config:
checkpoint_config = dict(interval=12, type='CheckpointHook')
log_config = dict(interval=10, hooks=[dict(type='TextLoggerHook')])
custom_hooks = [dict(type='NumClassCheckHook')]
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = 'checkpoints/yolov3_d53_mstrain-608_273e_coco_20210518_115020-a2c3acb8.pth'
resume_from = None
workflow = [('train', 1)]
model = dict(
    type='YOLOV3',
    backbone=dict(
        type='Darknet',
        depth=53,
        out_indices=(3, 4, 5),
        init_cfg=dict(type='Pretrained', checkpoint='open-mmlab://darknet53')),
    neck=dict(
        type='YOLOV3Neck',
        num_scales=3,
        in_channels=[1024, 512, 256],
        out_channels=[512, 256, 128]),
    bbox_head=dict(
        type='YOLOV3Head',
        num_classes=196,
        in_channels=[512, 256, 128],
        out_channels=[1024, 512, 256],
        anchor_generator=dict(
            type='YOLOAnchorGenerator',
            base_sizes=[[(116, 90), (156, 198), (373, 326)],
  

In [28]:
import shutil
from  os import walk

In [29]:
#get filenames from folder
def get_filenames(mypath):
  f = []
  for (dirpath, dirnames, filenames) in walk(mypath):
    f.extend(filenames)
    break
  return f

In [31]:
f=get_filenames("/content/data/car_ims/")

In [39]:
f

['005541.jpg',
 '008334.jpg',
 '009104.jpg',
 '013921.jpg',
 '011395.jpg',
 '002109.jpg',
 '001286.jpg',
 '006175.jpg',
 '009304.jpg',
 '014141.jpg',
 '003601.jpg',
 '012785.jpg',
 '009435.jpg',
 '009746.jpg',
 '007433.jpg',
 '010218.jpg',
 '013509.jpg',
 '003809.jpg',
 '015061.jpg',
 '003573.jpg',
 '002995.jpg',
 '009549.jpg',
 '001666.jpg',
 '012641.jpg',
 '003511.jpg',
 '008537.jpg',
 '015279.jpg',
 '003770.jpg',
 '005790.jpg',
 '013992.jpg',
 '008074.jpg',
 '011540.jpg',
 '007817.jpg',
 '005590.jpg',
 '011525.jpg',
 '015060.jpg',
 '005387.jpg',
 '012788.jpg',
 '001086.jpg',
 '011719.jpg',
 '010680.jpg',
 '009022.jpg',
 '003019.jpg',
 '001784.jpg',
 '005072.jpg',
 '012197.jpg',
 '001557.jpg',
 '001093.jpg',
 '013082.jpg',
 '002906.jpg',
 '009962.jpg',
 '015380.jpg',
 '014106.jpg',
 '009074.jpg',
 '001242.jpg',
 '015936.jpg',
 '015342.jpg',
 '012199.jpg',
 '000529.jpg',
 '007055.jpg',
 '000210.jpg',
 '011632.jpg',
 '003406.jpg',
 '016113.jpg',
 '005070.jpg',
 '009054.jpg',
 '000767.j

In [42]:
if "011171.jpg" in f:
  print("True")

True


In [20]:

!mv "/content/data/instances_test.json" "/content/data/annotations/"
!mv "/content/data/instances_val.json" "/content/data/annotations/"
!mv "/content/data/instances_train.json" "/content/data/annotations/"


In [68]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector


# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


/content/mmdetection/mmdet/core/anchor/builder.py:16: UserWarning: ``build_anchor_generator`` would be deprecated soon, please use ``build_prior_generator`` 
  '``build_anchor_generator`` would be deprecated soon, please use '
2021-07-12 17:40:34,221 - mmdet - INFO - load checkpoint from checkpoints/yolov3_d53_mstrain-608_273e_coco_20210518_115020-a2c3acb8.pth
2021-07-12 17:40:34,223 - mmdet - INFO - Use load_from_local loader


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


2021-07-12 17:40:34,409 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for bbox_head.convs_pred.0.weight: copying a param with shape torch.Size([255, 1024, 1, 1]) from checkpoint, the shape in current model is torch.Size([603, 1024, 1, 1]).
size mismatch for bbox_head.convs_pred.0.bias: copying a param with shape torch.Size([255]) from checkpoint, the shape in current model is torch.Size([603]).
size mismatch for bbox_head.convs_pred.1.weight: copying a param with shape torch.Size([255, 512, 1, 1]) from checkpoint, the shape in current model is torch.Size([603, 512, 1, 1]).
size mismatch for bbox_head.convs_pred.1.bias: copying a param with shape torch.Size([255]) from checkpoint, the shape in current model is torch.Size([603]).
size mismatch for bbox_head.convs_pred.2.weight: copying a param with shape torch.Size([255, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([603, 256, 1, 1]).
size mismatch for bbox_head.convs_pred

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1618/1618, 22.2 task/s, elapsed: 73s, ETA:     0s

...

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1618/1618, 22.7 task/s, elapsed: 71s, ETA:     0s

2021-07-12 19:26:36,968 - mmdet - INFO - Evaluating bbox...
2021-07-12 19:26:36,973 - mmdet - ERROR - The testing results of the whole dataset is empty.
2021-07-12 19:26:37,028 - mmdet - INFO - Epoch(val) [36][1618]	


Loading and preparing results...


2021-07-12 19:26:48,059 - mmdet - INFO - Epoch [37][10/203]	lr: 2.500e-03, eta: 11:03:46, time: 1.101, data_time: 0.306, memory: 8481, loss_cls: nan, loss_conf: nan, loss_xy: nan, loss_wh: nan, loss: nan, grad_norm: nan
2021-07-12 19:26:56,104 - mmdet - INFO - Epoch [37][20/203]	lr: 2.500e-03, eta: 11:03:36, time: 0.805, data_time: 0.014, memory: 8481, loss_cls: nan, loss_conf: nan, loss_xy: nan, loss_wh: nan, loss: nan, grad_norm: nan
2021-07-12 19:27:03,422 - mmdet - INFO - Epoch [37][30/203]	lr: 2.500e-03, eta: 11:03:22, time: 0.732, data_time: 0.013, memory: 8481, loss_cls: nan, loss_conf: nan, loss_xy: nan, loss_wh: nan, loss: nan, grad_norm: nan
2021-07-12 19:27:11,049 - mmdet - INFO - Epoch [37][40/203]	lr: 2.500e-03, eta: 11:03:09, time: 0.763, data_time: 0.013, memory: 8481, loss_cls: nan, loss_conf: nan, loss_xy: nan, loss_wh: nan, loss: nan, grad_norm: nan
2021-07-12 19:27:19,352 - mmdet - INFO - Epoch [37][50/203]	lr: 2.500e-03, eta: 11:03:01, time: 0.830, data_time: 0.013,

KeyboardInterrupt: ignored